In [1]:
import re
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold

/anaconda2/envs/py36/lib/python3.6/site-packages/lightgbm/__init__.py:47: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
columns = list(range(64))
names = list(range(64))
names.append('label')
train = pd.read_csv('../data/train/0.csv', header=0, names=names)
tmp = pd.read_csv('../data/train/1.csv', header=0, names=names)
train = pd.concat([train, tmp], axis=0)
tmp = pd.read_csv('../data/train/2.csv', header=0, names=names)
train = pd.concat([train, tmp], axis=0)
tmp = pd.read_csv('../data/train/3.csv', header=0, names=names)
train = pd.concat([train, tmp], axis=0)
test = pd.read_csv('../data/test/test.csv', names=columns)
train = train.reset_index()
train.drop('index', axis=1, inplace=True)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,label
0,4.0,-2.0,-8.0,-1.0,-8.0,-15.0,-8.0,-19.0,0.0,-9.0,...,4.0,-24.0,12.0,-2.0,-6.0,-20.0,-26.0,-44.0,-27.0,0
1,12.0,5.0,5.0,2.0,12.0,1.0,89.0,17.0,-18.0,-3.0,...,-14.0,1.0,-7.0,-8.0,-18.0,-3.0,4.0,62.0,-6.0,0
2,18.0,-8.0,-6.0,-5.0,7.0,16.0,10.0,25.0,-6.0,4.0,...,8.0,34.0,8.0,5.0,-10.0,17.0,51.0,92.0,45.0,0
3,-1.0,-4.0,-6.0,-8.0,17.0,2.0,14.0,6.0,4.0,-1.0,...,-5.0,-8.0,0.0,-2.0,0.0,6.0,-4.0,-68.0,-5.0,0
4,3.0,11.0,1.0,-5.0,4.0,-11.0,-1.0,2.0,-8.0,-21.0,...,3.0,-7.0,-3.0,-1.0,-5.0,-6.0,-15.0,11.0,0.0,0


In [3]:
x_train = train[columns]
y_train = train['label']
x_test = test[columns]
num_labels = 4

params = {
    'num_leaves': 60,
    'min_data_in_leaf': 30,
    'objective': 'multiclass',
    'num_class': num_labels,
    'max_depth': -1,
    'learning_rate': 0.03,
    "min_sum_hessian_in_leaf": 6,
    "boosting": "gbdt",
    "feature_fraction": 0.9,
    "bagging_freq": 1,
    "bagging_fraction": 0.8,
    "bagging_seed": 11,
    "lambda_l1": 0.1,
    "verbosity": -1,
    "nthread": 15,
    'metric': 'multi_logloss',
    "random_state": 0,}
    # 'device': 'gpu' }


folds = KFold(n_splits=5, shuffle=True, random_state=0)
oof = np.zeros((x_train.shape[0], num_labels))
preds_prob = np.zeros((x_test.shape[0], num_labels))

## train and predict
feature_importance_df = pd.DataFrame()
for fold_, (trn_idx, val_idx) in enumerate(folds.split(x_train)):
    print("fold {}".format(fold_ + 1))
    trn_data = lgb.Dataset(x_train.iloc[trn_idx], label=y_train.iloc[trn_idx])
    val_data = lgb.Dataset(x_train.iloc[val_idx], label=y_train.iloc[val_idx])
#     trn_data = lgb.Dataset(x_train[trn_idx], label=y_train[trn_idx])
#     val_data = lgb.Dataset(x_train[val_idx], label=y_train[val_idx])

    clf = lgb.train(
        params,
        trn_data,
        valid_sets=[trn_data, val_data],
        num_boost_round = 10000,
        verbose_eval = 100,
        early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(x_train.iloc[val_idx], num_iteration=clf.best_iteration)

    preds_prob += clf.predict(x_test, num_iteration=clf.best_iteration) / folds.n_splits
    
# preds = np.argmax(preds_prob, axis=1)

fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.219158	valid_1's multi_logloss: 0.33058
[200]	training's multi_logloss: 0.0668168	valid_1's multi_logloss: 0.191028
[300]	training's multi_logloss: 0.0323947	valid_1's multi_logloss: 0.152031
[400]	training's multi_logloss: 0.0208681	valid_1's multi_logloss: 0.137182
[500]	training's multi_logloss: 0.0156428	valid_1's multi_logloss: 0.13043
[600]	training's multi_logloss: 0.0127188	valid_1's multi_logloss: 0.127166
[700]	training's multi_logloss: 0.0108715	valid_1's multi_logloss: 0.125078
[800]	training's multi_logloss: 0.00960584	valid_1's multi_logloss: 0.123262
[900]	training's multi_logloss: 0.00867146	valid_1's multi_logloss: 0.12244
[1000]	training's multi_logloss: 0.00796345	valid_1's multi_logloss: 0.121188
[1100]	training's multi_logloss: 0.00740826	valid_1's multi_logloss: 0.120432
[1200]	training's multi_logloss: 0.00696395	valid_1's multi_logloss: 0.119806
[1300]	trainin

[2200]	training's multi_logloss: 0.00502826	valid_1's multi_logloss: 0.11566
[2300]	training's multi_logloss: 0.00491936	valid_1's multi_logloss: 0.115627
[2400]	training's multi_logloss: 0.00482029	valid_1's multi_logloss: 0.115534
[2500]	training's multi_logloss: 0.00473299	valid_1's multi_logloss: 0.115455
Early stopping, best iteration is:
[2468]	training's multi_logloss: 0.00475944	valid_1's multi_logloss: 0.115409


In [4]:
preds = np.argmax(preds_prob, axis=1)
submission = pd.DataFrame({'id': range(len(preds)), 'pred': preds})
submission['id'] = submission['id'] + 1
submission.to_csv("../data/ml_submission.csv", index=False, header=False)
submission.head()

,id,pred
0,1,3
1,2,2
2,3,0
3,4,1
4,5,0
